# **IMBD2022 表面膜厚預測**

ProProcess

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

class process:
    def __init__(self):
        pass

    def shuffle(self,X,Y):
        X,Y=shuffle(X,Y, random_state=0)
        return X,Y

    def plot_predict_GT(self,GT,predict,RMSE,fig_save_folder,label_name):
        x_list=list()
        for index in range(GT.shape[0]):
            x_list.append(index)
        x_np=np.array(x_list)
        plt.plot(x_np, GT,'ro-')
        plt.plot(x_np, predict,'g*--')
        plt.grid(True)
        plt.title(label_name+':'+str(RMSE))
        plt.legend(['GT_'+label_name,'Predict_'+label_name])
        plt.xlabel('Index of X')
        plt.ylabel('Thickness')
        plt.savefig(fig_save_folder+'_'+label_name+'.png')
        plt.clf()
            
    def load_dataset(self,xlsx_path,train_mode=True):
        df = pd.read_excel(xlsx_path)
        if(not train_mode):
            df=df.drop([
                        'create_time',
                        'update_time'
                        ], axis=1)
        return df

    def fillnan(self,data,drop_nan=True):
        print('Toal number of NAN before replant:',data.isnull().sum())
        if(drop_nan):
            data = data.dropna()
        else:
            for head in data.columns:
                data[head]=data[head].fillna(data[head].mean())
        print('Toal number of NAN after replant:',data.isnull().sum())
        return data

    def Get_Data_and_label(self,data,label_col=[],train_mode=True):
        Y=None
        if(train_mode):
            Y=data[label_col].to_numpy()
            data=data.drop(label_col,axis=1)
        X=data
        return X,Y

    def Normalization(self,X):
        Normalize_X=preprocessing.scale(X)
        return Normalize_X

    def splitdata(self,X,Y):
        X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2)
        return X_train, X_val, y_train, y_val


    def MSE(self,model_name,predict_y_test,y_test,label_name,fig_save_folder):
        sum_MSE=0
        for i in range(predict_y_test.shape[1]):
            MSE= mean_squared_error(y_test[i], predict_y_test[i], squared=False)
            self.plot_predict_GT(GT=y_test[:,i],
                    predict=predict_y_test[:,i],
                    RMSE=MSE,
                    fig_save_folder=fig_save_folder,
                    label_name=label_name[i])
            sum_MSE=sum_MSE+MSE
            print(model_name+' '+label_name[i]+' test MSE:',MSE)
        return sum_MSE/6

MyModel

In [9]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
import pickle
import xgboost as xgb
class model:

    def __init__(self):
        self.process=process()

    def XGBoostmodel_model_train(self,X_train, X_val, y_train, y_val,label_name):
        xgboost=xgb.XGBRegressor(max_depth=3, n_estimators=300, n_jobs=2,
                           objectvie='reg:squarederror', booster='gbtree',
                           random_state=42, learning_rate=0.05)
        models = MultiOutputRegressor(xgboost)
        models=models.fit(X_train, y_train)
        pickle.dump(models, open('XGBoost', 'wb'))
        predict_y_val=models.predict(X_val)
        avgMSE=self.process.MSE('XGBoost',predict_y_val,y_val,label_name,'./val_XGBoost/')
        return avgMSE

    def XGBoostmodel_model_test(self,X_test,y_test,label_name):
        models=pickle.load(open('XGBoost', 'rb'))
        predict_y_test=models.predict(X_test)
        avgMSE=self.process.MSE('XGBoost',predict_y_test,y_test,label_name,'./test_XGBoost/')
        return avgMSE

    def SVM_model_train(self,X_train, X_val, y_train, y_val,label_name):
        svr  = SVR(kernel="rbf", C=200, gamma=0.1, epsilon=0.1)
        models =MultiOutputRegressor(svr) 
        models=models.fit(X_train, y_train)
        predict_y_val = models.predict(X_val)
        pickle.dump(models, open('SVM', 'wb'))
        avgMSE=self.process.MSE('SVM',predict_y_val,y_val,label_name,'./val_SVM/')
        return avgMSE

    def SVM_model_test(self,X_test,y_test,label_name):
        models=pickle.load(open('SVM', 'rb'))
        predict_y_test=models.predict(X_test)
        avgMSE=self.process.MSE('SVM',predict_y_test,y_test,label_name,'./test_SVM/')
        return avgMSE


    def RandomForest_model_train(self,X_train, X_val, y_train, y_val,label_name):
        regr = RandomForestRegressor(max_depth=3, random_state=0)
        models =MultiOutputRegressor(regr) 
        models=models.fit(X_train, y_train)
        pickle.dump(models, open('RandomForest', 'wb'))
        predict_y_val = models.predict(X_val)
        avgMSE=self.process.MSE('RandomForest',predict_y_val,y_val,label_name,'./val_RandomForest/')
        return avgMSE

    def RandomForest_model_test(self,X_test,y_test,label_name):
        models=pickle.load(open('RandomForest', 'rb'))
        predict_y_test=models.predict(X_test)
        avgMSE=self.process.MSE('RandomForest',predict_y_test,y_test,label_name,'./test_RandomForest/')
        return avgMSE

    def GaussianNB_model_train(self,X_train, X_val, y_train, y_val,label_name):
        regr = GaussianNB()
        models =MultiOutputRegressor(regr) 
        models=models.fit(X_train, y_train)
        predict_y_val = models.predict(X_val)
        pickle.dump(models, open('GaussianNB', 'wb'))
        avgMSE=self.process.MSE('GaussianNB',predict_y_val,y_val,label_name,'./val_GaussianNB/')
        return avgMSE

    def GaussianNB_model_test(self,X_test,y_test,label_name):
        models=pickle.load(open('GaussianNB', 'rb'))
        predict_y_test=models.predict(X_test)
        avgMSE=self.process.MSE('GaussianNB',predict_y_test,y_test,label_name,'./test_GaussianNB/')
        return avgMSE



main主程式

In [10]:
import os 
def main(models,train_mode=True):
    _model=model()
    _preprocess=process()

    if(train_mode):

        print('=================training===============')
        data=_preprocess.load_dataset('training.xlsx',train_mode)
        data=_preprocess.fillnan(data,drop_nan=False)
        label_name=['sensor_point5_i_value',
                    'sensor_point6_i_value',
                    'sensor_point7_i_value',
                    'sensor_point8_i_value',
                    'sensor_point9_i_value',
                    'sensor_point10_i_value',
                                    ]
        X,Y=_preprocess.Get_Data_and_label(data,label_col=label_name,train_mode=True)
        X_train, X_val, y_train, y_val=_preprocess.splitdata(X,Y)

        X_val_normalize=_preprocess.Normalization(X_val)
        X_train_normalize=_preprocess.Normalization(X_train)

        print('X_train:',X_train_normalize.shape)
        print('y_train:',y_train.shape)
        print('X_val:',X_val_normalize.shape)
        print('y_val:',y_val.shape)
   
        for model_name in models:
            isExist = os.path.exists('val_'+model_name)
            if not isExist:
              os.makedirs('val_'+model_name)

            if(model_name=='SVM'):
                avgMSE=_model.SVM_model_train(X_train_normalize, X_val_normalize, y_train, y_val,label_name)
                print('val:',model_name+',avgMSE:',avgMSE)

            elif(model_name=='GaussianNB'):
                avgMSE=_model.GaussianNB_model_train(X_train_normalize, X_val_normalize, y_train, y_val,label_name)
                print('val:'+model_name+',avgMSE:',avgMSE)

            elif(model_name=='XGBoost'):
                avgMSE=_model.XGBoostmodel_model_train(X_train_normalize,X_val_normalize , y_train, y_val,label_name)
                print('val:'+model_name+',avgMSE:',avgMSE)



            elif(model_name=='RandomForest'):
                avgMSE=_model.RandomForest_model_train(X_train_normalize, X_val_normalize, y_train, y_val,label_name)
                print('val:'+model_name+',avgMSE:',avgMSE)

    else:
        print('=================testing===============')
        data=_preprocess.load_dataset('testing.xlsx',train_mode)
        data_ans=_preprocess.load_dataset('testing_ans.xlsx')

        data=_preprocess.fillnan(data,drop_nan=False)
        label_name=['sensor_point5_i_value',
                    'sensor_point6_i_value',
                    'sensor_point7_i_value',
                    'sensor_point8_i_value',
                    'sensor_point9_i_value',
                    'sensor_point10_i_value',
                                    ]
        
        X,_=_preprocess.Get_Data_and_label(data,label_col=label_name,train_mode=False)
        _,Y=_preprocess.Get_Data_and_label(data_ans,label_col=label_name,train_mode=True)

        # X_shuffled,y_shuffled = _preprocess.shuffle(X,Y)
        X_test_normalize=_preprocess.Normalization(X)


        print('X_test',X_test_normalize.shape)
        print('y_test:',Y.shape)
   
        for model_name in models:
            isExist = os.path.exists('test_'+model_name)
            if not isExist:
              os.makedirs('test_'+model_name)
            if(model_name=='SVM'):
                avgMSE=_model.SVM_model_test(X_test_normalize,Y,label_name)
                print('test:'+model_name+',avgMSE:',avgMSE)
            elif(model_name=='GaussianNB'):
                avgMSE=_model.GaussianNB_model_test(X_test_normalize,Y,label_name)
                print('test:'+model_name+',MSE:',avgMSE)
            elif(model_name=='XGBoost'):
                avgMSE=_model.XGBoostmodel_model_test(X_test_normalize,Y,label_name)
                print('test:'+model_name+',avgMSE:',avgMSE)
            elif(model_name=='RandomForest'):
                avgMSE=_model.RandomForest_model_test(X_test_normalize,Y,label_name)
                print('test:'+model_name+',avgMSE:',avgMSE)


if __name__ == '__main__':
    models=['SVM','GaussianNB','XGBoost','RandomForest']
    main(models,train_mode=True)


=================training===============
Toal number of NAN before replant: sensor_point5_i_value    0
sensor_point6_i_value    0
sensor_point7_i_value    0
sensor_point8_i_value    0
sensor_point9_i_value    0
                        ..
env_rpi15_lux            0
env_rpi15_pm1            0
env_rpi15_pm10           0
env_rpi15_pm25           0
env_rpi15_temp           0
Length: 131, dtype: int64
Toal number of NAN after replant: sensor_point5_i_value    0
sensor_point6_i_value    0
sensor_point7_i_value    0
sensor_point8_i_value    0
sensor_point9_i_value    0
                        ..
env_rpi15_lux            0
env_rpi15_pm1            0
env_rpi15_pm10           0
env_rpi15_pm25           0
env_rpi15_temp           0
Length: 131, dtype: int64
X_train: (508, 125)
y_train: (508, 6)
X_val: (127, 125)
y_val: (127, 6)


/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


SVM sensor_point5_i_value test MSE: 16.139879192825212
SVM sensor_point6_i_value test MSE: 12.81297264932475
SVM sensor_point7_i_value test MSE: 5.4231766027281045


/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


SVM sensor_point8_i_value test MSE: 38.26587650379034
SVM sensor_point9_i_value test MSE: 16.903742993702572
SVM sensor_point10_i_value test MSE: 31.829501799519605
val: SVM,avgMSE: 20.22919162364843


...

/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RandomForest sensor_point5_i_value test MSE: 12.167879162028257
RandomForest sensor_point6_i_value test MSE: 10.113499244452854
RandomForest sensor_point7_i_value test MSE: 14.192474324274624


/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/vuthanh/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RandomForest sensor_point8_i_value test MSE: 26.20205272238711
RandomForest sensor_point9_i_value test MSE: 15.556644722548338
RandomForest sensor_point10_i_value test MSE: 13.548858374328871
val:RandomForest,avgMSE: 15.296901425003343


<Figure size 640x480 with 0 Axes>